In [2]:
import torch
import torch.nn as nn
from torch.nn import functional as F

torch.manual_seed(1337)

In [4]:
!wget -O input.txt https://cs.stanford.edu/people/karpathy/char-rnn/shakespeare_input.txt

# Read the file into a Python variable
with open("input.txt", "r", encoding="utf-8") as f:
    text = f.read()

--2025-03-01 10:40:46--  https://cs.stanford.edu/people/karpathy/char-rnn/shakespeare_input.txt
Resolving cs.stanford.edu (cs.stanford.edu)... 171.64.64.64
Connecting to cs.stanford.edu (cs.stanford.edu)|171.64.64.64|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4573338 (4.4M) [text/plain]
Saving to: ‘input.txt’

input.txt           100%[===================>]   4.36M  6.58MB/s    in 0.7s    

2025-03-01 10:40:47 (6.58 MB/s) - ‘input.txt’ saved [4573338/4573338]



In [5]:
print(text[:1000])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.

All:
We know't, we know't.

First Citizen:
Let us kill him, and we'll have corn at our own price.
Is't a verdict?

All:
No more talking on't; let it be done: away, away!

Second Citizen:
One word, good citizens.

First Citizen:
We are accounted poor citizens, the patricians good.
What authority surfeits on would relieve us: if they
would yield us but the superfluity, while it were
wholesome, we might guess they relieved us humanely;
but they think we are too dear: the leanness that
afflicts us, the object of our misery, is as an
inventory to particularise their abundance; our
sufferance is a gain to them Let us revenge this with
our pikes, ere we become rakes: for the gods know I
speak this in hunger for bread, not in thirst for revenge.



In [3]:
# Preprocessing of text and splitting in train and validation___________________

with open("input.txt", "r", encoding="utf-8") as f:
    text = f.read()

chars = sorted(list(set(text)))
vocab_size = len(chars)

stoi = {ch: i for i, ch in enumerate(chars)}
itos = {i: ch for i, ch in enumerate(chars)}

encode = lambda s: [stoi[ch] for ch in s]
decode = lambda l: ''.join([itos[i] for i in l])

data = torch.tensor(encode(text), dtype = int)

n = int(0.85 * len(data))
train_data = data[:n]
val_data = data[n:]

In [ ]:
# Hyperparameters_______________________________________________________________

batch_size = 64 # how many independent sequences will we process in parallel?
block_size = 256 # what is the maximum context length for predictions?
max_iters = 10000
eval_interval = 500
learning_rate = 1e-4
device = 'cuda' if torch.cuda.is_available() else 'cpu'
eval_iters = 500
n_embed = 66
n_head = 6
n_layer = 10
dropout = 0.3


In [ ]:
# Split data in mini-batches____________________________________________________

def get_batch_random(split):
    data = train_data if split == 'train' else val_data
    random_index = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i: i + block_size] for i in random_index])
    y = torch.stack([data[(i + 1): (i + 1) + block_size] for i in random_index])
    x, y = x.to(device), y.to(device)
    return x, y


@torch.no_grad()
def estimate_loss():
    model.eval()
    out = {}
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)

        for k in range(eval_iters):
            x, y = get_batch_random(split)
            logits, loss = model(x, y)
            losses[k] = loss.item()

        out[split] = losses.mean()


    model.train()
    return out

In [ ]:
class Head(nn.Module):
    def __init__(self, head_size):
        super().__init__()
        self.key = nn.Linear(n_embed, head_size, bias = False)
        self.query = nn.Linear(n_embed, head_size, bias = False)
        self.value = nn.Linear(n_embed, head_size, bias = False)
        self.register_buffer('tril', torch.tril(torch.ones(block_size, block_size)))

        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        B, T, C = x.shape

        k = self.key(x)
        q = self.query(x)
        wei = q @ k.transpose(-2, -1) * (C ** -0.5)
        wei = wei.masked_fill(self.tril[:T, :T] == 0, float('-inf'))
        wei = F.softmax(wei, dim = -1)
        v = self.value(x)
        out = wei @ v

        return out


class MultiHeadAttention(nn.Module):
    def __init__(self, num_heads, head_size):
        super().__init__()
        self.heads = nn.ModuleList([Head(head_size) for _ in range(num_heads)])
        self.projection = nn.Linear(n_embed, n_embed)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        out = torch.cat([h(x) for h in self.heads], dim = -1)
        out = self.projection(out)
        out = self.dropout(out)
        return out


class FeedForward(nn.Module):
    def __init__(self, n_embed):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(n_embed, 4 * n_embed),
            nn.ReLU(),
            nn.Linear(4 * n_embed, n_embed),
            nn.Dropout(dropout)
        )

    def forward(self, x):
        return self.net(x)


class Block(nn.Module):
    def __init__(self, n_embed, n_head):
        super().__init__()
        head_size = n_embed // n_head
        self.self_attention_head = MultiHeadAttention(n_head, head_size) # 4 heads of 8-dimensional self-attention
        self.fforward = FeedForward(n_embed)
        self.ln1 = nn.LayerNorm(n_embed)
        self.ln2 = nn.LayerNorm(n_embed)

    def forward(self, x):
        x = x + self.self_attention_head(self.ln1(x))
        x = x + self.fforward(self.ln2(x))
        return x



class BigramLanguageModel(nn.Module):
    def __init__(self, vocab_size):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, n_embed)
        self.position_embedding_table = nn.Embedding(block_size, n_embed)
        # self.self_attention_head = Head(n_embed)
        # self.self_attention_head = MultiHeadAttention(4, n_embed // 4) # 4 heads of 8-dimensional self-attention
        # self.fforward = FeedForward(n_embed)
        self.blocks = nn.Sequential(*[Block(n_embed, n_head = n_head) for _ in range(n_layer)])
        # self.blocks = nn.Sequential(
        #     Block(n_embed, n_head = 4),
        #     Block(n_embed, n_head = 4),
        #     Block(n_embed, n_head = 4),
        # )
        self.ln_f = nn.LayerNorm(n_embed)
        self.language_model_head = nn.Linear(n_embed, vocab_size)

    def forward(self, idx, targets = None):
        B, T = idx.shape

        token_embeddings = self.token_embedding_table(idx) #(Batch, Time, Channel)

        position_embeddings = self.position_embedding_table(torch.arange(T, device = idx.device)) #(Time, Channel)
        # position_embeddings = self.position_embedding_table(
        #     torch.arange(min(T, block_size), device=idx.device)
        # )

        x = token_embeddings + position_embeddings

        # Positional embeddings (ensure shape matches token embeddings)
        # T_clamped = min(T, block_size)  # Ensure T does not exceed block_size
        # position_embeddings = self.position_embedding_table(
        #     torch.arange(T_clamped, device=idx.device)
        # ).unsqueeze(0)  # Add batch dimension -> (1, T_clamped, n_embed)

        # # Ensure positional embeddings match token embeddings
        # token_embeddings[:, :T_clamped, :] += position_embeddings

        # x = self.self_attention_head(x)
        # x = self.fforward(x)

        x = self.blocks(x)
        x = self.ln_f(x)
        logits = self.language_model_head(x) #(Batch, Time, vocab_size)

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B * T, C)
            targets = targets.view(B * T)

            loss = F.cross_entropy(logits, targets) # quality of logits based on targets

        return logits, loss

    def generate(self, idx, max_new_tokens):
        # idx -> (B, T)
        with torch.no_grad():
            for i in range(max_new_tokens):
                idx_cond = idx[:, -block_size:] # (B, T)
                logits, loss = self(idx_cond) # (B, T, C)
                logits = logits[:, -1, :] # last time step only | becomes (B, C)
                prob = F.softmax(logits, dim = -1)
                idx_next = torch.multinomial(prob, num_samples = 1) # predicted | (B, 1)
                idx = torch.cat((idx, idx_next), dim = 1) # (B, T + 1)
        return idx


model = BigramLanguageModel(vocab_size)
model = model.to(device)

In [ ]:
optimizer = torch.optim.AdamW(model.parameters(), lr = 1e-3)


for iter in range(max_iters):
    if (iter % eval_interval == 0):
        losses = estimate_loss()
        print(f"step: {iter}, train loss: {losses['train']}, val loss: {losses['val']:.4f}")

    x_batch, y_batch = get_batch_random('train')
    x_batch, y_batch = x_batch.to(device), y_batch.to(device)

    logits, loss = model(x_batch, y_batch)
    optimizer.zero_grad(set_to_none = True)
    loss.backward()
    optimizer.step()

print(loss.item())
